# Method 1: DeepSeek API Extraction

## Overview
Cost-effective poster metadata extraction using DeepSeek API. This approach offers the best balance of accuracy and cost for most applications.

## Accuracy Note
The 85-90% accuracy estimate is unvalidated - based on limited testing only. Actual accuracy must be determined through proper Cochran sampling validation before production use.

## Performance Characteristics
- **Estimated Accuracy**: 85-90% (unvalidated - requires Cochran sampling validation)
- **Cost**: ~$0.003 per poster (200x cheaper than GPT-4)
- **Speed**: 5-15 seconds per poster
- **Hallucination Risk**: Low-Medium (mitigated by structured prompts)
- **Setup**: Easy - just requires API key

## Best For
- Production systems with budget constraints
- High-volume processing
- Quick prototyping and development

In [1]:
# Imports and setup
import os
import json
import fitz  # PyMuPDF
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Any
import time
from openai import OpenAI
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

# Load environment variables from .env file
load_dotenv()

print("✅ Imports successful")
print("🎯 Method 1: DeepSeek API Extraction")

✅ Imports successful
🎯 Method 1: DeepSeek API Extraction


In [2]:
# Configuration and functions
CONFIG = {
    'model': 'deepseek-chat',
    'base_url': 'https://api.deepseek.com/v1',
    'max_tokens': 2000,
    'temperature': 0.1,
    'cost_per_1m_tokens': 0.14
}

def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text from PDF using PyMuPDF"""
    doc = fitz.open(pdf_path)
    text = ""
    
    for page_num, page in enumerate(doc):
        page_text = page.get_text()
        if page_text:
            text += f"\n--- Page {page_num + 1} ---\n{page_text}"
    
    doc.close()
    return text.strip()

def create_extraction_prompt(text: str) -> str:
    """Create enhanced structured prompt for metadata extraction"""
    return f"""You are a scientific metadata extraction expert. Extract structured information from this poster text with high precision.

POSTER TEXT:
{text[:2500]}...

EXTRACTION INSTRUCTIONS:
1. Look for title in ALL CAPS or large text at the top
2. Find all author names (often with superscript numbers for affiliations)  
3. Identify institutional affiliations (usually below authors)
4. Extract 6-8 specific keywords from methods and results sections
5. Summarize key findings concisely
6. Find funding acknowledgments (often at bottom)

Return ONLY valid JSON in this exact format:
{{
  "title": "exact poster title as written",
  "authors": [
    {{"name": "Full Name", "affiliations": ["University/Institution"], "email": "email@domain.com or null"}}
  ],
  "summary": "2-sentence summary of research objective and main finding",
  "keywords": ["specific", "technical", "terms", "from", "poster", "content"],
  "methods": "detailed methodology description from poster",
  "results": "quantitative results and key findings with numbers if present",
  "references": [
    {{"title": "paper title", "authors": "author names", "year": 2024, "journal": "journal name"}}
  ],
  "funding_sources": ["specific funding agency or grant numbers"],
  "conference_info": {{"location": "city, country", "date": "date range"}}
}}

Be precise and thorough. Extract only information explicitly present in the text."""

print("✅ Configuration and functions defined")

✅ Configuration and functions defined


In [3]:
def extract_with_deepseek(text: str, api_key: str) -> Dict:
    """Extract metadata using DeepSeek API"""
    client = OpenAI(
        api_key=api_key,
        base_url=CONFIG['base_url']
    )
    
    prompt = create_extraction_prompt(text)
    
    response = client.chat.completions.create(
        model=CONFIG['model'],
        messages=[
            {"role": "system", "content": "You are a scientific text extraction assistant. Return only valid JSON."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=CONFIG['max_tokens'],
        temperature=CONFIG['temperature']
    )
    
    content = response.choices[0].message.content.strip()
    
    # Clean response
    if content.startswith('```json'):
        content = content[7:-3].strip()
    elif content.startswith('```'):
        content = content[3:-3].strip()
    
    return json.loads(content)

def extract_poster_metadata(pdf_path: str) -> Dict[str, Any]:
    """Complete extraction pipeline"""
    start_time = time.time()
    
    print(f"📄 Processing: {Path(pdf_path).name}")
    
    # Extract text
    text = extract_text_from_pdf(pdf_path)
    print(f"📏 Extracted {len(text)} characters")
    
    # Check API key
    api_key = os.getenv('DEEPSEEK_API_KEY')
    

    # Extract with DeepSeek
    print("🤖 Extracting with DeepSeek API...")
    metadata = extract_with_deepseek(text, api_key)
    
    # Add processing metadata
    processing_time = time.time() - start_time
    metadata['extraction_metadata'] = {
        'timestamp': datetime.now().isoformat(),
        'processing_time': processing_time,
        'method': 'deepseek_api',
        'model': 'deepseek-chat',
        'estimated_cost': (len(text) + 1000) / 1000000 * 0.14,
        'text_length': len(text)
    }
    
    return metadata


In [4]:
# Run extraction
pdf_path = "/home/joneill/poster_project/data/test-poster.pdf"

if Path(pdf_path).exists():
    print("🚀 Running Method 1: DeepSeek API Extraction")
    print("=" * 60)
    
    results = extract_poster_metadata(pdf_path)
    
    # Display results
    print(f"\n📄 TITLE: {results['title']}")
    print(f"👥 AUTHORS: {len(results['authors'])} found")
    for author in results['authors']:
        print(f"   • {author['name']}")
    
    print(f"\n📝 SUMMARY: {results['summary'][:100]}...")
    print(f"🔑 KEYWORDS: {', '.join(results['keywords'][:5])}")
    print(f"⏱️  Processing time: {results['extraction_metadata']['processing_time']:.2f}s")
    
    # Show cost estimate
    if 'estimated_cost' in results['extraction_metadata']:
        print(f"💰 Estimated cost: ${results['extraction_metadata']['estimated_cost']:.4f}")
    
    # Save results
    output_path = Path("output/method1_deepseek_results.json")
    output_path.parent.mkdir(exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"💾 Results saved to: {output_path}")
    print("✅ Method 1 completed successfully!")
    
else:
    print("❌ Test poster not found")

🚀 Running Method 1: DeepSeek API Extraction
📄 Processing: test-poster.pdf
📏 Extracted 3731 characters
🤖 Extracting with DeepSeek API...

📄 TITLE: INFLUENCE OF DRUG-POLYMER INTERACTIONS ON RELEASE KINETICS OF PLGA AND PLA/PEG NPS
👥 AUTHORS: 5 found
   • Merve Gul
   • Ida Genta
   • Maria M. Perez Madrigal
   • Carlos Aleman
   • Enrica Chiesa

📝 SUMMARY: This research investigates drug-polymer interactions to develop novel drug delivery strategies again...
🔑 KEYWORDS: PLGA nanoparticles, PLA/PEG micelles, microfluidic synthesis, curcumin delivery, drug-polymer interactions
⏱️  Processing time: 33.80s
💰 Estimated cost: $0.0007
💾 Results saved to: output/method1_deepseek_results.json
✅ Method 1 completed successfully!
